In [38]:
type Direction = | Up | Down | Left | Right

type Move = 
    { 
        Direction : Direction
        Steps : int 
    }

type Rope = 
    {
        Head : int*int
        Tail : int*int
        TailVisited : (int*int) list
    }

module Rope =
    let starting = { Head = (0,0); Tail = (0,0); TailVisited = [(0,0)] }

    let moveHead dir rope =
        let (x,y) = rope.Head
        match dir with
        | Up -> { rope with Head = (x, y + 1) }
        | Down -> { rope with Head = (x, y - 1) }
        | Left -> { rope with Head = (x - 1, y) }
        | Right -> { rope with Head = (x + 1, y) }        

    let moveTail rope =
        let (hx, hy) = rope.Head
        let (tx, ty) = rope.Tail
        let (dx, dy) = (hx-tx, hy-ty)
        if (abs dx <= 1) && (abs dy <= 1) then
            rope // ends are next to each other
        else
            let t' = (tx + sign dx, ty + sign dy) // move one space max
            { rope with Tail = t'; TailVisited = t'::rope.TailVisited }

    let rec applyMove move rope =
        if move.Steps = 0 then
            rope
        else
            rope
            |> moveHead move.Direction
            |> moveTail
            |> applyMove { move with Steps = move.Steps-1 }

    let applyMoves moves rope =
        moves
        |> Seq.fold (fun r m -> applyMove m r) rope

    let visitedCount rope =
        rope.TailVisited
        |> List.distinct
        |> List.length


In [39]:
let parseLine (s : string) =
    let steps = int (s.Substring(2))
    let dir = 
        match s[0] with
        | 'U' -> Up
        | 'D' -> Down
        | 'L' -> Left
        | 'R' -> Right
        | _ -> failwith "Unrecognised direction"
    { Direction = dir; Steps = steps }


In [40]:
#r "nuget: FsUnit"

open FsUnitTyped

let testInput = 
    [|
        "R 4"
        "U 4"
        "L 3"
        "D 1"
        "R 4"
        "D 1"
        "L 5"
        "R 2"
    |]
    |> Array.map parseLine

let testResult = Rope.starting |> Rope.applyMoves testInput

Rope.visitedCount testResult |> shouldEqual 13

Installed Packages FsUnit, 5.1.0

In [41]:
open System.IO

let sourcePath = Path.Combine(__SOURCE_DIRECTORY__, "input_09.txt")
let moves = 
    File.ReadAllLines(sourcePath)
    |> Array.map parseLine

let result = 
    Rope.starting
    |> Rope.applyMoves moves
    |> Rope.visitedCount

In [ ]:
printfn "Tail visited %d spaces" result